In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Loading CSV

In [0]:
n_images = 1

In [0]:
import pandas as pd 
df=pd.read_csv('/content/gdrive/My Drive/ Data_Entry_2017.csv')
df2 = df[['Image Index','Finding Labels']]
df2 = df2[0:n_images][:]

import cv2
images = []

imgfolder = '/content/gdrive/My Drive/images/'
for i in range(n_images):
  img = df2['Image Index'][i]
  path = imgfolder + img
  temp = cv2.imread(path)
  im = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
  print(im.shape)
  #images.append(im)


(1024, 1024)
(1024, 1024)
(1024, 1024)
(1024, 1024)
(1024, 1024)
(1024, 1024)
(1024, 1024)
(1024, 1024)


# Loading Images

In [0]:
#images = pickle.load(open("/content/gdrive/My Drive/DataFiles/images_2k.pickle",'rb'))

### GLCM

In [0]:
import math
def idm(g):
  r, c = g.shape
  idm = 0.0
  for i in range(r):
    for j in range(c):
      idm += (1/(1+(i-j)**2)) * g[i][j]
  return idm

def variance(g):
  r, c = g.shape
  var = 0.0
  for i in range(r):
    for j in range(c):
      var += ((i-j)**2)*g[i][j]
  return var

def sumaver(g, rowsum, colsum):
  P = rowsum + colsum
  sa = 0.0
  for i in range(len(P)):
    sa += i*P[i]
  return sa

def sumentr(g, rowsum, colsum):
  P = rowsum + colsum
  se = 0.0
  for i in range(len(P)):
    if(P[i]>0):
      se -= P[i]*(math.log(P[i]))

  return se

def diffentr(g, rowsum, colsum):
  P = rowsum-colsum
  de=0.0
  for i in range(len(P)):
    if(P[i]>0):
      de -= P[i]*(math.log(P[i]))
  return de

def inertia(g):
  r, c = g.shape
  iner = 0.0
  for i in range(r):
    for j in range(c):
      iner += ((i-j)**2)*g[i][j]
  return iner

def clus_shade_prom(g, rowmean, colmean):
  shade = 0.0
  prom = 0.0
  r, c = g.shape
  for i in range(r):
    for j in range(c):
      shade += ((i+j-rowmean-colmean)**3)*g[i][j]
      prom += ((i+j-rowmean-colmean)**4)*g[i][j]
  return shade, prom

def entropy(g):
  r,c = g.shape
  entr = 0.0
  for i in range(r):
    for j in range(c):
      if(g[i][j]!=0):
        entr += g[i][j] * (math.log(g[i][j]))
  return -1*entr

In [0]:
import skimage.feature
import numpy as np
contrast=[0]*len(images)
homogen=[0]*len(images)
energy=[0]*len(images)
correl=[0]*len(images)
dissim=[0]*len(images)
asm=[0]*len(images)
indm=[0]*len(images)
var=[0]*len(images)
sumav=[0]*len(images)
sument=[0]*len(images)
dent=[0]*len(images)
iner=[0]*len(images)
shade=[0]*len(images)
prom=[0]*len(images)
entr=[0]*len(images)
maxprob=[0]*len(images)
rowsums=[0]*len(images)
colsums=[0]*len(images)

for i in range(len(images)):
  g = skimage.feature.greycomatrix(images[i], [1], [0], levels=256, symmetric=False, normed=True)
  
  contrast[i] = skimage.feature.greycoprops(g, 'contrast')[0][0]
  homogen[i] = skimage.feature.greycoprops(g, 'homogeneity')[0][0]
  energy[i] = skimage.feature.greycoprops(g, 'energy')[0][0]
  correl[i] = skimage.feature.greycoprops(g, 'correlation')[0][0]
  dissim[i] = skimage.feature.greycoprops(g, 'dissimilarity')[0][0]
  asm[i] = skimage.feature.greycoprops(g, 'ASM')[0][0]
  r, c, d, t = g.shape
  g = np.reshape(g, [r, c])

  rowsums[i] = np.sum(g, axis=1)
  colsums[i] = np.sum(g, axis=0)

  rowmean = 0
  colmean = 0
  assert r==c

  rowmean = np.sum(j*rowsums[i][j] for j in range(r))
  colmean = np.sum(j*colsums[i][j] for j in range(c))

  indm[i] = idm(g)
  var[i] = variance(g)
  sumav[i] = sumaver(g, rowsums[i], colsums[i])
  sument[i] = sumentr(g, rowsums[i], colsums[i])
  dent[i] = diffentr(g, rowsums[i], colsums[i])
  iner[i] = inertia(g)
  s, p = clus_shade_prom(g, rowmean, colmean)
  shade[i] = s
  prom[i] = p
  entr[i] = entropy(g)
  maxprob[i] = np.max(g)
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


### Adding features to dataframe

In [0]:
df2['Contrast']=contrast
df2['Homogeneity']=homogen
df2['Energy']=energy
df2['Correlation']=correl
df2['Dissimilarity']=dissim
df2['ASM']=asm
df2['Inverse Difference Moment']=indm
df2['Variance']=var
df2['Sum Averange']=sumav
df2['Sum Entropy']=sument
df2['Difference Entropy']=dent
df2['Inertia']=iner
df2['Cluster Shade']=shade
df2['Cluster Prominence']=prom
df2['Entropy']=entr
df2['Maximum Probability']=maxprob

In [0]:
labelset = df2['Finding Labels']
labels=[]
for l in labelset: 
  s = l.split('|')  #spliting to get the list of labels associated with each instance
  for item in s:
    labels.append(item)  #add the labels so obtained to the list of labels
labels=set(labels)       #making the list a set, to get unique values
#print(labels)

In [0]:
#creating a separate column in the data frame for each label
for x in labels:
  df2[x] = [0]*len(images)

In [0]:
#to assign 0/1 to each label column, if occurring in that image or not
for a in range(len(images)):
  label_set = df2['Finding Labels'][a].split('|')
  for l in label_set:
    df2[l][a] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


### Outlier detection and removal

In [0]:
#too many outliers
'''
df_features = df2[df2.columns[2:18]]  #extracting textural features data

q1 = df_features.quantile(0.25) #first quantile
q3 = df_features.quantile(0.75) #third quantile
iqr = q3-q1   #inter quartile range
#print(iqr)

#indices of rows that have a feature that is an outlier
indices = df_features.index[~((df_features<(q1-1.5*iqr)) | (df_features>(q3+1.5*iqr))).any(axis=1)].tolist()
#print(indices)

outliers = [] #List of outliers indices

for i in range(0,n_images):
  if i in indices:
    pass
  else:
    outliers.append(i)
    #print(df2.as_matrix()[i])
  
print("Number of outliers = ", len(outliers))
final_df = df2.iloc[indices]
'''

'\ndf_features = df2[df2.columns[2:18]]  #extracting textural features data\n\nq1 = df_features.quantile(0.25) #first quantile\nq3 = df_features.quantile(0.75) #third quantile\niqr = q3-q1   #inter quartile range\n#print(iqr)\n\n#indices of rows that have a feature that is an outlier\nindices = df_features.index[~((df_features<(q1-1.5*iqr)) | (df_features>(q3+1.5*iqr))).any(axis=1)].tolist()\n#print(indices)\n\noutliers = [] #List of outliers indices\n\nfor i in range(0,n_images):\n  if i in indices:\n    pass\n  else:\n    outliers.append(i)\n    #print(df2.as_matrix()[i])\n  \nprint("Number of outliers = ", len(outliers))\nfinal_df = df2.iloc[indices]\n'

### Normalization

In [0]:
# def normalize(df):
#     result = df.copy()
#     for feature_name in df.columns:
#         if(feature_name == 'Image Index' or feature_name == 'Finding Labels'):
#             continue
#         max_value = df[feature_name].max()
#         min_value = df[feature_name].min()
#         result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
#     return result

In [0]:
# #df2 = final_df.copy()   #uncomment if running outliers
# df2 = normalize(df2)
# df2.head()

,Image Index,Finding Labels,Contrast,Homogeneity,Energy,Correlation,Dissimilarity,ASM,Inverse Difference Moment,Variance,Sum Averange,Sum Entropy,Difference Entropy,Inertia,Cluster Shade,Cluster Prominence,Entropy,Maximum Probability,Cardiomegaly,Pleural_Thickening,Mass,Consolidation,Emphysema,Hernia,Edema,Atelectasis,Pneumonia,No Finding,Pneumothorax,Infiltration,Fibrosis,Effusion,Nodule
0,00000001_000.png,Cardiomegaly,0.157161,0.118460,0.009259,0.940113,0.729588,0.001023,0.118460,0.157161,0.429485,1.000000,0.912781,0.157161,0.639580,0.398948,0.975513,0.026390,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00000001_001.png,Cardiomegaly|Emphysema,0.177691,0.161897,0.015554,0.922323,0.686759,0.001807,0.161897,0.177691,0.409876,0.990397,0.858806,0.177691,0.635436,0.376602,0.947608,0.035337,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00000001_002.png,Cardiomegaly|Effusion,0.149438,0.559867,0.054394,0.946321,0.296949,0.008215,0.559867,0.149438,0.754751,0.875932,0.569339,0.149438,0.275717,0.654725,0.678704,0.026147,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,00000002_000.png,No Finding,0.058930,0.347642,0.037501,0.945213,0.410669,0.005095,0.347642,0.058930,0.863079,0.762436,0.676350,0.058930,0.559121,0.178133,0.718266,0.010421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,00000003_000.png,Hernia,0.226332,0.169383,0.010204,0.922012,0.694920,0.001136,0.169383,0.226332,0.406516,0.994460,0.736545,0.226332,0.674902,0.473449,0.949091,0.020424,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
df2.head()

,Image Index,Finding Labels,Contrast,Homogeneity,Energy,Correlation,Dissimilarity,ASM,Inverse Difference Moment,Variance,Sum Averange,Sum Entropy,Difference Entropy,Inertia,Cluster Shade,Cluster Prominence,Entropy,Maximum Probability,Hernia,Pneumothorax,Edema,Atelectasis,Cardiomegaly,Nodule,Infiltration,Pleural_Thickening,Mass,Effusion,Pneumonia,Fibrosis,Consolidation,No Finding,Emphysema
0,00000001_000.png,Cardiomegaly,13.624845,0.356682,0.025027,0.998504,2.479109,0.000626,0.356682,13.624845,256.271761,9.516242,0.010311,13.624845,-3.710118e+05,6.204582e+08,7.965408,0.011879,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,Cardiomegaly|Emphysema,15.113981,0.377159,0.027408,0.998231,2.370910,0.000751,0.377159,15.113981,251.569510,9.485354,0.009701,15.113981,-3.934828e+05,5.865152e+08,7.882999,0.015439,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,00000001_002.png,Cardiomegaly|Effusion,13.064677,0.564775,0.042099,0.998599,1.386116,0.001772,0.564775,13.064677,334.268845,9.117170,0.006431,13.064677,-2.344133e+06,1.008983e+09,7.088873,0.011783,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
3,00000002_000.png,No Finding,6.499827,0.464725,0.035710,0.998582,1.673413,0.001275,0.464725,6.499827,360.245327,8.752106,0.007640,6.499827,-8.073136e+05,2.850427e+08,7.205706,0.005526,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,00000003_000.png,Hernia,18.642068,0.380688,0.025385,0.998226,2.391526,0.000644,0.380688,18.642068,250.763886,9.498423,0.008320,18.642068,-1.794676e+05,7.336246e+08,7.887377,0.009506,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df2.to_csv('/content/gdrive/My Drive/DataFiles/basicGlcm_2k.csv')